Primeiro instalamos as bibliotecas faiss-cpu para busca vetorial e criação de banco vetorial local, sentence-transformers para transformar os dados textuais em numéricos via embeddings, crewai para orquestração do nosso fluxo agentes.

In [1]:
!pip install faiss-cpu sentence-transformers crewai 'crewai[tools]'

Abaio são importadas as bibliotecas pandas e numpy para manipulação de dados,  as classes do CrewAI Agent para criar agente, Task para criar tarefas para os agentes, Crew para orquestrar o fluxo, Process para definir a ordem do fluxo e LLM para usar modelos alternativos como o Gemini 2 Flash, SentenceTransformer para embeddings e fais para indexação vetorial. Já crewai.tools é para a criação de tools para nossos agentes e crewai_tools é para usar ferramentas já prontas disponíveis como é o caso do SerperDevTool para realizar pesquisas na web.

In [2]:
import pandas as pd
import numpy as np
import os
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from crewai_tools import SerperDevTool
from google.colab import userdata
from IPython.display import display, Markdown
from sentence_transformers import SentenceTransformer
import faiss
import json

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("website_url")
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/selenium_scraping_tool/selenium_scraping_tool.py:26: PydanticDeprecatedSince20: Pydantic V1 style `@validator`

Carrega o conjunto de dados de transcrições médicas do arquivo CSV 'mtsamples.csv' baixado no kaglle que contém exemplos médicos.

In [3]:
dados=pd.read_csv('/content/mtsamples.csv')

In [4]:
dados

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."
...,...,...,...,...,...,...
4994,4994,Patient having severe sinusitis about two to ...,Allergy / Immunology,Chronic Sinusitis,"HISTORY:, I had the pleasure of meeting and e...",NaN
4995,4995,This is a 14-month-old baby boy Caucasian who...,Allergy / Immunology,Kawasaki Disease - Discharge Summary,"ADMITTING DIAGNOSIS: , Kawasaki disease.,DISCH...","allergy / immunology, mucous membranes, conjun..."
4996,4996,A female for a complete physical and follow u...,Allergy / Immunology,Followup on Asthma,"SUBJECTIVE: , This is a 42-year-old white fema...",NaN
4997,4997,Mother states he has been wheezing and coughing.,Allergy / Immunology,Asthma in a 5-year-old,"CHIEF COMPLAINT: , This 5-year-old male presen...",NaN


In [5]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         4999 non-null   int64 
 1   description        4999 non-null   object
 2   medical_specialty  4999 non-null   object
 3   sample_name        4999 non-null   object
 4   transcription      4966 non-null   object
 5   keywords           3931 non-null   object
dtypes: int64(1), object(5)
memory usage: 234.5+ KB


Realizando a limpeza dos dados removendo as colunas que não serão utilizadas ('keywords' e 'Unnamed: 0') e também eliminando linhas com valores nulos.

In [6]:
dados = dados.drop('keywords', axis=1)
dados = dados.drop('Unnamed: 0', axis=1)
dados = dados.dropna()
dados.info()
dados

<class 'pandas.core.frame.DataFrame'>
Index: 4966 entries, 0 to 4998
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   description        4966 non-null   object
 1   medical_specialty  4966 non-null   object
 2   sample_name        4966 non-null   object
 3   transcription      4966 non-null   object
dtypes: object(4)
memory usage: 194.0+ KB


,description,medical_specialty,sample_name,transcription
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr..."
1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb..."
2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ..."
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit..."
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...
...,...,...,...,...
4994,Patient having severe sinusitis about two to ...,Allergy / Immunology,Chronic Sinusitis,"HISTORY:, I had the pleasure of meeting and e..."
4995,This is a 14-month-old baby boy Caucasian who...,Allergy / Immunology,Kawasaki Disease - Discharge Summary,"ADMITTING DIAGNOSIS: , Kawasaki disease.,DISCH..."
4996,A female for a complete physical and follow u...,Allergy / Immunology,Followup on Asthma,"SUBJECTIVE: , This is a 42-year-old white fema..."
4997,Mother states he has been wheezing and coughing.,Allergy / Immunology,Asthma in a 5-year-old,"CHIEF COMPLAINT: , This 5-year-old male presen..."


In [7]:
processado = pd.DataFrame(columns=['processado'])
processado['processado'] = dados.apply(lambda row: {
    "description": str(row['description']),
    "medical_specialty": str(row['medical_specialty']),
    "sample_name": str(row['sample_name']),
    "transcription": str(row['transcription'])
}, axis=1)

Inicializando o SentenceTransformer para embeddings com o modelo all-MiniLM-L6-v2 que possui 384 dimensões e configurando as keys armazenadas no Colab.

In [73]:
model = SentenceTransformer('all-MiniLM-L6-v2')
os.environ['SERPER_API_KEY'] = userdata.get('SERPER-EI')
gemini_api_key = userdata.get('google_pago')
llm = LLM(model='gemini/gemini-2.0-flash',api_key=gemini_api_key)
search_tool = SerperDevTool()

Cria o índice de busca vetorial FAISS gerando embeddings para cada linha do processado e os adicionando ao índice com métrica de distância L2 para usar depois com busca de similaridade.

In [9]:
embeddings = model.encode(processado['processado'].tolist())
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings, dtype=np.float32))

Abaixo temos a @tool search_medical_documents que será integrada com CrewAI, ela recebe a descrição dos sintomas e o número de resultados desejados, no código está fixado em 5. Primeiro recebe a entrada depois rejeita descrições vazias ou placeholders, depois converte a descrição textual em um vetor de embeddings usando o all-MiniLM-L6-v2. Após esse processo é feita uma busca vetorial com index.search() que retorna distâncias e índices dos documentos mais similares, retornando as as linhas das colunas do df (descrição, especialidade médica, nome da amostra) que mais se assemelham com as informadas inicialmente e manda em JSON o output.

In [85]:
@tool
def search_medical_documents(symptoms_description: str, top_k: int = 5) -> str:
    """Busca documentos médicos com base na descrição dos sintomas.
    """
    try:
        if not symptoms_description or "[Please" in symptoms_description:
            return json.dumps({"error": "Por favor, forneça uma descrição detalhada dos sintomas reais do paciente."})
        query_vector = model.encode([symptoms_description])[0]
        query_vector_np = np.array([query_vector], dtype=np.float32)
        distances, indices = index.search(query_vector_np, top_k)

        results = []
        for idx, dist in zip(indices[0], distances[0]):
            if idx >= 0 and idx < len(dados):
                results.append({
                    "índice": int(idx),
                    "distância": float(dist),
                    "descrição": dados.iloc[idx]['description'],
                    "especialidade": dados.iloc[idx]['medical_specialty'],
                    "nome_amostra": dados.iloc[idx]['sample_name'],
                    "transcrição_resumida": dados.iloc[idx]['transcription'][:200] + "..."
                })

        return json.dumps(results, ensure_ascii=False, indent=2)
    except Exception as e:
        return f"Erro na busca: {str(e)}"

Abaixo temos o agent_diagnostico que é um especialista médico, que têm como objetivo identificar condições médicas através de análise comparativa, temos também verbose=True que mostra os logs, memory=True mantêm contexto entre interações e uma história de fundo. Depois em tools temos de importante a tool search_medical_documents para busca semântica. Já na tarefa task_diagnostico fala que deve ser feita a tradução português-inglês já que a base de dados vetorial está em inglês e a tarefa espera um relatório médico esperado incluindo diagnóstico, justificativa e recomendação.

In [91]:
agent_diagnostico = Agent(
    role="Especialista em Diagnóstico Médico",
    goal="Identificar com precisão a condição do paciente através de análise comparativa de casos médicos similares",
    verbose=True,
    memory=True,
    backstory="""Você é um especialista em diagnóstico médico com anos de experiência clínica.
    Você elaborou uma metodologia única com base nos seus conhecimentos médicos e de análise de dados
    que combina conhecimento médico tradicional com análise de dados avançada.
    Sua abordagem meticulosa inclui análise de sintomas, histórico do paciente e comparação
    com casos semelhantes para determinar diagnósticos com alta precisão.""",
    tools=[search_medical_documents],
    llm=llm,
    allow_delegation=True
)

task_diagnostico = Task(
    description="""

    IMPORTANTE: Nossa base de dados médicos está em inglês. Quando receber uma descrição de sintomas em português, você deve:

    1. Traduzir os sintomas do paciente do português para o inglês
    2. Utilizar a tradução em inglês como entrada para a ferramenta search_medical_documents
    3. Analisar os resultados da busca
    4. Formular seu diagnóstico e recomendações em português

    Exemplo:
    - Sintomas em português: "febre alta, tosse seca, falta de ar e dor no corpo"
    - Tradução para busca: "high fever, dry cough, shortness of breath and body pain"
    - Use esta tradução com a ferramenta search_medical_documents
    - Apresente seu diagnóstico final em português

    Analise detalhadamente os sintomas do paciente e utilize a ferramenta de busca vetorial
    para comparar com casos médicos semelhantes. Use a descrição textual dos sintomas
    como entrada para a ferramenta search_medical_documents.

    Considere:
    1. A correspondência entre os sintomas do paciente e os casos encontrados
    2. A frequência com que cada diagnóstico aparece nos resultados
    3. A gravidade potencial das condições identificadas
    4. Possíveis diagnósticos diferenciais

    Forneça um diagnóstico.
    """,
    expected_output="""
    Um relatório detalhado contendo:
    - Diagnóstico
    - Justificativa baseada nos sintomas e casos similares
    - Recomendação sobre a urgência de atendimento médico
    """,
    tools=[search_medical_documents],
    agent=agent_diagnostico
)

Abaixo temos o agent_tratamento que é um especialista em planos de tratamento médico que foi configurado da mesma forma do agent_diagnostico. A diferença em relação ao agente anterior é que esse utiliza search_tool para pesquisar web. A task_tratamento do agente fala para ele fazer todas as suas operações em ingl6es e traduzir ao final, ele deve criar um plano incluindo tratamentos, medicamentos com dosagens, opções não-farmacológicas, mudanças no estilo de vida e tempo estimado de recuperação. A saída esperada é um plano estruturado com essas informações, referências médicas confiáveis e um disclaimer sobre ser gerado por IA.

In [92]:
agent_tratamento = Agent(
    role="Especialista em Planos de Tratamento Médico",
    goal="Desenvolver um plano de tratamento personalizado, baseado em evidências científicas atualizadas",
    verbose=True,
    memory=True,
    backstory="""Você é um renomado especialista em planos de tratamento médico com vasta experiência em
    medicina baseada em evidências. Com doutorado em farmacologia clínica e especialização em medicina
    integrativa, você desenvolveu protocolos de tratamento adotados por grandes instituições de saúde.
    Sua abordagem combina os tratamentos convencionais mais eficazes com práticas complementares
    suportadas por pesquisas científicas, sempre priorizando a segurança e bem-estar do paciente.""",
    tools=[search_tool],
    llm=llm,
    allow_delegation=True
)

task_tratamento = Task(
    description="""

    IMPORTANTE: Ao desenvolver planos de tratamento:

    1. Use o inglês quando pesquisar diretrizes e tratamentos médicos na internet
    2. Mesmo que o diagnóstico esteja em português, faça suas pesquisas usando termos médicos em inglês
    3. Traduza as informações relevantes encontradas do inglês para o português
    4. Apresente o plano de tratamento final em português para manter a consistência com o diagnóstico

    Com base no diagnóstico fornecido pelo Especialista em Diagnóstico Médico, pesquise
    e desenvolva um plano de tratamento abrangente. Inclua:

    1. Tratamentos de primeira linha recomendados por diretrizes médicas atuais
    2. Opções medicamentosas com dosagens típicas (quando aplicável)
    3. Tratamentos não-farmacológicos recomendados
    4. Mudanças no estilo de vida e dieta que podem auxiliar na recuperação
    5. Tempo estimado para recuperação

    Cite fontes médicas confiáveis e atualizadas para suas recomendações.
    """,
    expected_output="""
    Um plano de tratamento estruturado contendo:
    - Resumo das opções de tratamento para o diagnóstico identificado
    - Recomendações medicamentosas (quando aplicável) com dosagens típicas
    - Intervenções não-farmacológicas recomendadas
    - Mudanças no estilo de vida e dieta
    - Referências a fontes médicas confiáveis (pelo menos 3)
    - Disclaimer médico sobre o fato de essa ser uma resposta gerada por IA
    """,
    tools=[search_tool],
    agent=agent_tratamento
)

Abaixo temos Crew que orquestra os agentes em sequência (diagnóstico seguido de tratamento) e define funções para coletar sintomas e gerar vetores de embeddings e converte.

In [88]:
crew = Crew(
    agents=[agent_diagnostico, agent_tratamento],
    tasks=[task_diagnostico, task_tratamento],
    verbose=True,
    process=Process.sequential)

# Função para coletar os sintomas do paciente
def coletar_sintomas():
    print("Por favor, descreva os problemas/sintomas atuais do paciente:")
    descricao = input("Descrição dos sintomas: ")
    return descricao

def gerar_vetor(descricao):
    vetor = model.encode([descricao])
    return vetor[0].tolist()

A seguir temos a função diagnostico que coordena todo o processo, coletando sintomas do usuário e armazenando em descricao_sintomas, executando o Crew em sequência e exibindo o relatório médico final com diagnóstico e plano de tratamento.

In [94]:
def diagnostico():
    print("Por favor, descreva os problemas/sintomas atuais do paciente:")
    descricao_sintomas = input("Descrição dos sintomas: ")
    inputs = {
        'symptoms_description': descricao_sintomas,
        'top_k': 5}
    print("\nProcessando informações médicas... Por favor aguarde.\n")
    result = crew.kickoff(inputs=inputs)


    print("\n===== RELATÓRIO MÉDICO =====\n")
    print(result)

Abaixo temos um exemplo de uso onde inserimos a condição da paciente: Ela está se sentindo realmente desgastada. Além das fortes dores de cabeça que mencionamos, ela também tem sofrido com náuseas constantes e uma grande sensibilidade à luz e ao som. Esses episódios têm sido cada vez mais frequentes, impactando sua rotina diária. Ela percebe que, quando está em um ambiente mais iluminado ou com muito barulho, a dor piora, tornando difícil até mesmo se concentrar no trabalho ou nas atividades do dia a dia. A sensação de cansaço é constante, e ela se vê frequentemente forçada a interromper suas atividades para descansar, já que a dor a incapacita de seguir em frente. Além disso, ela tem notado que esses momentos costumam ocorrer quando está estressada ou cansada, principalmente após noites de sono ruim, o que agrava ainda mais a condição. E em seguida recebemos o relatório completo com possível problema indentificado, tratamentos, tempo de recuperação, fontes e o aviso de que foi gerado por uma IA.

In [95]:
diagnostico()

Por favor, descreva os problemas/sintomas atuais do paciente:
Descrição dos sintomas: Ela está se sentindo realmente desgastada. Além das fortes dores de cabeça que mencionamos, ela também tem sofrido com náuseas constantes e uma grande sensibilidade à luz e ao som. Esses episódios têm sido cada vez mais frequentes, impactando sua rotina diária. Ela percebe que, quando está em um ambiente mais iluminado ou com muito barulho, a dor piora, tornando difícil até mesmo se concentrar no trabalho ou nas atividades do dia a dia. A sensação de cansaço é constante, e ela se vê frequentemente forçada a interromper suas atividades para descansar, já que a dor a incapacita de seguir em frente. Além disso, ela tem notado que esses momentos costumam ocorrer quando está estressada ou cansada, principalmente após noites de sono ruim, o que agrava ainda mais a condição

Processando informações médicas... Por favor aguarde.

# Agent: Especialista em Diagnóstico Médico
## Task: 
    Analise detalhadamente